In [1]:
import numpy as np

In [2]:
def grid2str(grid):
    # converts a 2d numpy array to a string representation
    grid_str = str([list(row) for row in grid])
    grid_str = grid_str.replace(', ', '')
    grid_str = grid_str.replace('[[', '|')
    grid_str = grid_str.replace('][', '|')
    grid_str = grid_str.replace(']]', '|')
    return grid_str

### Generating grids that change color

In [15]:
class ChangeColor:
    
    def __init__(self):
        pass

    def generate_caption(self, input_color, output_color):
        color_words = {0: 'black',
                       1: 'navy',
                       2: 'red',
                       3: 'green',
                       4: 'yellow',
                       5: 'grey',
                       6: 'pink',
                       7: 'orange',
                       8: 'blue',
                       9: 'brown'}
        input_color_word = color_words[input_color]
        output_color_word = color_words[output_color]
        # captions = [f'change the color of the {input_color_word} object to {output_color_word}',
        #             f'change the color of the {input_color_word} shape to {output_color_word}',
        #             f'set the color of the {input_color_word} object to {output_color_word}',
        #             f'set the color of the {input_color_word} shape to {output_color_word}',
        #             f'change the {input_color_word} tiles to {output_color_word}',
        #             f'change the {input_color_word} squares to {output_color_word}',
        #             f'change all of the {input_color_word} tiles to {output_color_word}',
        #             f'change all of the {input_color_word} squares to {output_color_word}']
        captions = [f'change {input_color_word} to {output_color_word}']
        caption = np.random.choice(captions).split(' ')
        return caption
    def generate_io_pair(self, input_color, output_color):
        width = np.random.randint(2, 5)
        height = np.random.randint(2, 5)

        # create empty grid
        input_grid = np.zeros((height, width), dtype=int)
        output_grid = np.zeros((height, width), dtype=int)
        # add object to grid
        obj = np.ones((2, 2)) * input_color
        obj_x = np.random.randint(0, input_grid.shape[1] - 1)
        obj_y = np.random.randint(0, input_grid.shape[0] - 1)
        input_grid[obj_y:obj_y+obj.shape[1], obj_x:obj_x+obj.shape[0]] = input_color
        output_grid[obj_y:obj_y+obj.shape[1], obj_x:obj_x+obj.shape[0]] = output_color
        return input_grid, output_grid
    def generate_episode(self, ns, nq):
        # ns: number of support examples
        # nq: number of query examples
        # lists to store episode info
        support_inputs = []
        support_outputs = []
        query_inputs = []
        query_outputs = []
        captions = []
        caption_length = []
        # sample colors
        input_color = np.random.randint(1, 10)
        output_color = input_color
        while output_color == input_color:
            output_color = np.random.randint(1, 10)
        # if train:
        #     input_color = np.random.randint(1, 10)
        #     output_color = input_color
        #     while output_color == input_color:
        #         if input_color % 2 == 0:
        #             output_color = np.random.choice([2, 4, 6, 8])
        #         else:
        #             output_color = np.random.choice([1, 3, 5, 7, 9])
        # else:
        #     input_color = np.random.randint(1, 10)
        #     output_color = input_color
        #     while output_color == input_color:
        #         if input_color % 2 == 0:
        #             output_color = np.random.choice([1, 3, 5, 7, 9])                
        #         else:
        #             output_color = np.random.choice([2, 4, 6, 8])
        # sample caption
        caption = self.generate_caption(input_color, output_color)
        caption_length = len(caption)
        for i in range(ns):
            input_grid, output_grid = self.generate_io_pair(input_color, output_color)
            support_inputs.append(grid2str(input_grid))
            support_outputs.append(grid2str(output_grid))
        for i in range(nq):
            input_grid, output_grid = self.generate_io_pair(input_color, output_color)
            query_inputs.append(grid2str(input_grid))
            query_outputs.append(grid2str(output_grid))
        episode = {'support_inputs': support_inputs,
                   'support_outputs': support_outputs,
                   'query_inputs': query_inputs,
                   'query_outputs': query_outputs,
                   'caption': caption,
                   'caption_length': caption_length}
        return episode

In [16]:
ns = 5
nq = 5
task = ChangeColor()
episode = task.generate_episode(ns, nq)

In [17]:
print(episode)

{'support_inputs': ['|6600|6600|', '|66|66|00|00|', '|00|66|66|', '|00|00|66|66|', '|66|66|00|00|'], 'support_outputs': ['|1100|1100|', '|11|11|00|00|', '|00|11|11|', '|00|00|11|11|', '|11|11|00|00|'], 'query_inputs': ['|66|66|', '|000|066|066|', '|66|66|00|00|', '|66|66|00|', '|00|66|66|'], 'query_outputs': ['|11|11|', '|000|011|011|', '|11|11|00|00|', '|11|11|00|', '|00|11|11|'], 'caption': ['change', 'pink', 'to', 'navy'], 'caption_length': 4}


### Generating grids that move

In [21]:
class Move:
    
    def __init__(self):
        pass
    def generate_caption(self, direction):

        captions = [f'the block moved {direction}',
                    f'move {direction}',
                    f'the block moves {direction}',
                    f'go {direction}',
                    f'the square moved {direction}',
                    f'the square moves {direction}',
                    f'the block goes {direction}',
                    f'the square goes {direction}']

        caption = np.random.choice(captions).split(' ')
        return caption
    def generate_io_pair(self, direction):
        width = np.random.randint(2, 5)
        height = np.random.randint(2, 5)
        width = 6
        height = 6

        # create empty grid
        input_grid = np.zeros((height, width), dtype=int)
        output_grid = np.zeros((height, width), dtype=int)
        # add object to grid
        obj = np.ones((2, 2))
        if direction == "down":
            x, y = np.random.choice(range(2)), np.random.choice(range(5))
            x_, y_ = x+3,y
        elif direction == "up":
            x, y = np.random.choice(range(3,5)), np.random.choice(range(5))
            x_, y_ = x - 3, y
        elif direction == "right":
            x, y = np.random.choice(range(5)), np.random.choice(range(2))
            x_, y_ = x,y + 3
        elif direction == "left":
            x, y = np.random.choice(range(5)), np.random.choice(range(3,5))
            x_, y_ = x,y - 3

        input_grid[x:x+2, y:y+2] = 1
        output_grid[x_:x_+2, y_:y_+2] = 1
        return input_grid, output_grid
    def generate_episode(self, ns, nq):
        # ns: number of support examples
        # nq: number of query examples
        # lists to store episode info
        support_inputs = []
        support_outputs = []
        query_inputs = []
        query_outputs = []
        captions = []
        caption_length = []
        # sample colors
        directions = ["up", "down", "left", "right"]
        direction = np.random.choice(directions)

        caption = self.generate_caption(direction)
        caption_length = len(caption)

        for i in range(ns):
            input_grid, output_grid = self.generate_io_pair(direction)
            support_inputs.append(grid2str(input_grid))
            support_outputs.append(grid2str(output_grid))
        for i in range(nq):
            input_grid, output_grid = self.generate_io_pair(direction)
            query_inputs.append(grid2str(input_grid))
            query_outputs.append(grid2str(output_grid))

        episode = {'support_inputs': support_inputs,
                   'support_outputs': support_outputs,
                   'query_inputs': query_inputs,
                   'query_outputs': query_outputs,
                   'caption': caption,
                   'caption_length': caption_length}
        return episode

In [22]:
ns = 5
nq = 1
task = Move()
episode = task.generate_episode(ns, nq)

[[0 0 0 0 0 0]
 [0 1 1 0 0 0]
 [0 1 1 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 1 1 0 0 0]
 [0 1 1 0 0 0]]
[[1 1 0 0 0 0]
 [1 1 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 0 0 0 0]
 [1 1 0 0 0 0]
 [0 0 0 0 0 0]]
[[0 0 0 0 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 0]]
[[0 0 0 0 0 0]
 [0 0 0 1 1 0]
 [0 0 0 1 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 1 1 0]
 [0 0 0 1 1 0]]
[[0 0 0 0 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 0]]


In [23]:
print(episode)

{'support_inputs': ['|000000|011000|011000|000000|000000|000000|', '|110000|110000|000000|000000|000000|000000|', '|000011|000011|000000|000000|000000|000000|', '|000000|000110|000110|000000|000000|000000|', '|000011|000011|000000|000000|000000|000000|'], 'support_outputs': ['|000000|000000|000000|000000|011000|011000|', '|000000|000000|000000|110000|110000|000000|', '|000000|000000|000000|000011|000011|000000|', '|000000|000000|000000|000000|000110|000110|', '|000000|000000|000000|000011|000011|000000|'], 'query_inputs': ['|000000|001100|001100|000000|000000|000000|'], 'query_outputs': ['|000000|000000|000000|000000|001100|001100|'], 'caption': ['the', 'square', 'moves', 'down'], 'caption_length': 4}
